In [ ]:
import torch
import numpy as np
import os

from dataloader.ae_data_handler import AEDataHandler
from models.multi_VAE import MultiVAE
from loss.vae_loss import VAELoss
from metric.recall_at_k import RecallAtK
from metric.revenue_at_k import RevenueAtK
from trainer import Trainer

from hyperopt import pyll, fmin, tpe, hp, Trials, STATUS_OK, STATUS_FAIL
import pprint

# import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument("--data_dir", default="/home/blagoj/working_dir/data/", help="the path to the directory where the data is stored")
# parser.add_argument("--models_dir", default="/home/blagoj/working_dir/models", help="the path to the directory where to save the models, it must be empty")
# args = parser.parse_args()

# # get the arguments
# dir_path = args.data_dir
# save_to_path = args.models_dir
dir_path = "/home/blagoj/working_dir/data/"
save_to_path_parent = "/home/blagoj/working_dir/models"

# set up logging
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger("main")
logger.setLevel(logging.INFO)

# set cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_path = os.path.join(
    dir_path, "movielens_small_training.npy")
validation_input_data_path = os.path.join(
    dir_path, "movielens_small_validation_input.npy")
validation_output_data_path = os.path.join(
    dir_path, "movielens_small_validation_test.npy")
test_input_data_path = os.path.join(
    dir_path, "movielens_small_test_input.npy")
test_output_data_path = os.path.join(
    dir_path, "movielens_small_test_test.npy")
products_data_path = os.path.join(
    dir_path, "movielens_products_data.npy")

data_handler = AEDataHandler(
    "MovieLensSmall", train_data_path, validation_input_data_path,
    validation_output_data_path, test_input_data_path,
    test_output_data_path)

input_dim = data_handler.get_input_dim()
output_dim = data_handler.get_output_dim()

products_data_np = np.load(products_data_path)
products_data_torch = torch.tensor(
    products_data_np, dtype=torch.float32).to(device)

In [ ]:
def train_model(space):
    params_model = {
        "dropout": 0.5,
        "no_latent_features": 200,
        "norm_mean": 0.0,
        "norm_std": 0.001,
        "input_size": 8936,
        "output_size": 8936,
        "enc1_out": 600,
        "enc2_in": 600,
        "enc2_out": 400,
        "dec1_in": 200,
        "dec1_out": 600,
        "dec2_in": 600
    }
    
    params_trainer = {
        "seed": 42,
        "normalize_gradients": True,
        "learning_rate": 1e-3,
        "batch_size_training": 500,
        "shuffle_training": True,
        "drop_last_batch_training": True,
        "batch_size_validation": 500,
        "shuffle_validation": True,
        "drop_last_batch_validation": False,
        "batch_size_testing": 500,
        "shuffle_testing": True,
        "drop_last_batch_testing": True,
        "number_of_epochs": 10,
        "frank_wolfe_max_iter": 100,
        "anneal": True,
        "beta_start": 0,
        "beta_cap": 0.3,
        "beta_step": "0.3/10000"
    }
    
    params_model["dropout"] = space["drop_out"]
    params_trainer["learning_rate"] = space["learning_rate"]
    
    space_str = ""
    for key in space:
        space_str += "KEY{},VALUE{}-".format(key, space[key])
    space_str = space_str[:-1]
    save_to_path = os.path.join(save_to_path_parent, space_str)
    
    model = MultiVAE(params=params_model)

    correctness_loss = VAELoss()
    revenue_loss = VAELoss(weighted_vector=products_data_torch)
    losses = [correctness_loss, revenue_loss]

    recallAtK = RecallAtK(k=10)
    revenueAtK = RevenueAtK(k=10, revenue=products_data_np)
    validation_metrics = [recallAtK, revenueAtK]

    trainer = Trainer(data_handler, model, losses, validation_metrics, save_to_path, params=params_trainer)
    val_loss = trainer.train()
    result = {
        "loss": val_loss,
        "space": space,
        "status": STATUS_OK
    }
    return result

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

space = {
    'learning_rate': hp.loguniform('learning_rate', -7, -4),
    'drop_out': hp.uniform('drop_out', 0.0, 0.7)
}

trials = Trials()

best = fmin(
    fn=train_model,
    space=space,
    algo=tpe.suggest,
    trials=trials,
    max_evals=10
)

print("Found minimum after 10 trials:")
print(best)
print("")

print("The space and results of the 3 first trials:")
pp.pprint(trials.trials[0])
pp.pprint(trials.trials[1])
pp.pprint(trials.trials[2])
print("")

print("The 'result' key of each trial (only firs 3):")
pp.pprint(trials.trials[0]["result"])
pp.pprint(trials.trials[1]["result"])
pp.pprint(trials.trials[2]["result"])